In [5]:
import tensorflow as tf  
import numpy as np  
import glob  
import random  
from PIL import Image   
import configparser  
import os  

IMAGE_HEIGHT = 36  
IMAGE_WIDTH = 48  
MAX_CAPTCHA = 4 
CHAR_SET_LEN = 11 


####################################################################  
#读热编码
def Onehot(text):
    vector = list()
    lentext = len(text)
    for i in range(lentext):
         vector.append(np.zeros(4 * 11))
    for i,s in enumerate(text):
        s = str(s)
        for index,n in enumerate (s):
            vector[i][(index)*11+int(n)] = 1
        if index < 4:
            for m in range(index+1,4):
                vector[i][(m)*11+10] = 1
                
    
    return vector


# 定义CNN  
def crack_captcha_cnn(X, keep_prob, w_alpha=0.01, b_alpha=0.1):  
    x = tf.reshape(X, shape=[-1, IMAGE_HEIGHT, IMAGE_WIDTH, 1])  
    # print(x.get_shape())  

    # w_c1_alpha = np.sqrt(2.0/(IMAGE_HEIGHT*IMAGE_WIDTH)) #  
    # w_c2_alpha = np.sqrt(2.0/(3*3*32))  
    # w_c3_alpha = np.sqrt(2.0/(3*3*64))  
    # w_d1_alpha = np.sqrt(2.0/(8*32*64))  
    # out_alpha = np.sqrt(2.0/1024)  

    # 3 conv layer  
    w_c1 = tf.Variable(w_alpha * tf.random_normal([3, 3, 1, 32]))  
    b_c1 = tf.Variable(b_alpha * tf.random_normal([32]))  
    conv1 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(x, w_c1, strides=[1, 1, 1, 1], padding='VALID'), b_c1))  
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  
    conv1 = tf.nn.dropout(conv1, keep_prob)  
    # print(conv1.get_shape())  

    w_c2 = tf.Variable(w_alpha * tf.random_normal([3, 3, 32, 64]))  
    b_c2 = tf.Variable(b_alpha * tf.random_normal([64]))  
    conv2 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(conv1, w_c2, strides=[1, 1, 1, 1], padding='VALID'), b_c2))  
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  
    conv2 = tf.nn.dropout(conv2, keep_prob)  
    # print(conv2.get_shape())  

    w_c3 = tf.Variable(w_alpha * tf.random_normal([3, 3, 64, 64]))  
    b_c3 = tf.Variable(b_alpha * tf.random_normal([64]))  
    conv3 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(conv2, w_c3, strides=[1, 1, 1, 1], padding='VALID'), b_c3))  
    conv3 = tf.nn.max_pool(conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  
    conv3 = tf.nn.dropout(conv3, keep_prob) 
    # print(conv3.get_shape())  

    # Fully connected layer  
    w_d = tf.Variable(w_alpha * tf.random_normal([3*5*64, 1024]))  
    b_d = tf.Variable(b_alpha * tf.random_normal([1024]))  
    dense = tf.reshape(conv3, [-1, w_d.get_shape().as_list()[0]])  
    dense = tf.nn.relu(tf.add(tf.matmul(dense, w_d), b_d))  
    dense = tf.nn.dropout(dense, keep_prob)  

    w_out = tf.Variable(w_alpha * tf.random_normal([1024, MAX_CAPTCHA * CHAR_SET_LEN]))  
    b_out = tf.Variable(b_alpha * tf.random_normal([MAX_CAPTCHA * CHAR_SET_LEN]))  
    out = tf.add(tf.matmul(dense, w_out), b_out)  
    # out = tf.nn.softmax(out)  
    return out  
 

In [6]:

def read_and_decode(filename):
    #根据文件名生成一个队列
    filename_queue = tf.train.string_input_producer([filename])

    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)   #返回文件名和文件
    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'label': tf.FixedLenFeature([], tf.int64),
                                           'img_raw' : tf.FixedLenFeature([], tf.string),
                                       })

    img = tf.decode_raw(features['img_raw'], tf.uint8)
    img = tf.cast(img,tf.float64)

    img = tf.reshape(img, [36, 48, 1])
#     img = tf.split(img,3,2)[0]
    label = tf.cast(features['label'], tf.int32)
    img_batch, label_batch = tf.train.shuffle_batch([img, label],
                                                batch_size=64, capacity=2000,
                                                min_after_dequeue=1000)
    return img_batch,label_batch


In [15]:
X = tf.placeholder(tf.float32, [None, IMAGE_HEIGHT , IMAGE_WIDTH,1])  
Y = tf.placeholder(tf.float32, [None, MAX_CAPTCHA * CHAR_SET_LEN])  
keep_prob = tf.placeholder(tf.float32)  # dropout  
output = crack_captcha_cnn(X, keep_prob) 
predicty = tf.reshape(output, [-1, MAX_CAPTCHA, CHAR_SET_LEN])
predictY = tf.reshape(Y, [-1, MAX_CAPTCHA, CHAR_SET_LEN])
# loss  
# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output, Y))  
loss = 0
for i in range(4):
    loss = loss + ((0.25)*tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits( labels=predictY[:,i], logits=predicty[:,i])))  
# 最后一层用来分类的softmax和sigmoid有什么不同？  
# optimizer 为了加快训练 learning_rate应该开始大，然后慢慢衰  
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss)  

predict = tf.reshape(output, [-1, MAX_CAPTCHA, CHAR_SET_LEN])  
max_idx_p = tf.argmax(predict, 2)  
max_idx_l = tf.argmax(tf.reshape(Y, [-1, MAX_CAPTCHA, CHAR_SET_LEN]), 2)  
correct_pred = tf.equal(max_idx_p, max_idx_l)  
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))  
saver = tf.train.Saver(max_to_keep=3)#保存模型 
max_acc = 100
batch_1x, batch_1y = read_and_decode("train_1.tfrecord")
batch_2x, batch_2y = read_and_decode("train_2.tfrecord")
batch_3x, batch_3y = read_and_decode("train_3.tfrecord")
batch_4x, batch_4y = read_and_decode("train_4.tfrecord")
batch_5x, batch_5y = read_and_decode("train_5.tfrecord")
batch_6x, batch_6y = read_and_decode("train_6.tfrecord")
batch_7x, batch_7y = read_and_decode("train_7.tfrecord")
batch_8x, batch_8y = read_and_decode("train_8.tfrecord")
     
batch_x_v, batch_y_v = read_and_decode("validation.tfrecord")
    
with tf.Session() as sess:  
    sess.run(tf.global_variables_initializer())  
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        
        
    for i in range(3200):
        if i<400:
            val, l = sess.run([batch_1x,batch_1y])
        elif i<800:
            val, l = sess.run([batch_2x,batch_2y])
        elif i<1200:
            val, l = sess.run([batch_3x,batch_3y])
        elif i<1600:
            val, l = sess.run([batch_4x,batch_4y])
        elif i<2000:
            val, l = sess.run([batch_5x,batch_5y])
        elif i<2400:
            val, l = sess.run([batch_6x,batch_6y])
        elif i<2800:
            val, l = sess.run([batch_7x,batch_7y])
        elif i<3200:
            val, l = sess.run([batch_8x,batch_8y])
                
        l = Onehot(l)
        acc = sess.run(accuracy, feed_dict={X: val, Y: l, keep_prob: 0.7})
        if i%10 == 0:
            print("step is:" + str(i), "acc is :" + str(acc))  
            
        if i % 50 == 0:  
            val_v,l_v =sess.run([batch_x_v,batch_y_v])
            l_v = Onehot(l_v)
            _, lossSize = sess.run([optimizer, loss], feed_dict={X: val_v, Y: l_v, keep_prob: 1.0})
            print("step is:" + str(i), u"损失函数大小为" + str(lossSize))  
                
            if lossSize < max_acc:
                max_acc = lossSize
                saver.save(sess,'model/mnist.ckpt',global_step=i+1)
    coord.request_stop()
    coord.join(threads)


step is:0 acc is :0.0585938
step is:0 损失函数大小为2.40462
step is:10 acc is :0.234375
step is:20 acc is :0.300781
step is:30 acc is :0.199219
step is:40 acc is :0.246094
step is:50 acc is :0.222656
step is:50 损失函数大小为2.38357
step is:60 acc is :0.390625
step is:70 acc is :0.355469
step is:80 acc is :0.375
step is:90 acc is :0.296875
step is:100 acc is :0.28125
step is:100 损失函数大小为2.35995
step is:110 acc is :0.398438
step is:120 acc is :0.421875
step is:130 acc is :0.335938
step is:140 acc is :0.34375
step is:150 acc is :0.414062
step is:150 损失函数大小为2.33199
step is:160 acc is :0.378906
step is:170 acc is :0.402344
step is:180 acc is :0.371094
step is:190 acc is :0.335938
step is:200 acc is :0.410156
step is:200 损失函数大小为2.28228
step is:210 acc is :0.429688
step is:220 acc is :0.421875
step is:230 acc is :0.40625
step is:240 acc is :0.398438
step is:250 acc is :0.4375
step is:250 损失函数大小为2.213
step is:260 acc is :0.421875
step is:270 acc is :0.410156
step is:280 acc is :0.410156
step is:290 acc is :

In [16]:
batch_x_t, batch_y_t = read_and_decode("test.tfrecord")

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())  
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    #saver.restore(sess, tf.train.latest_checkpoint('./model/'))
    model_file=tf.train.latest_checkpoint('model/')
    saver.restore(sess,model_file)
        
    for i in range(40):  
        x_t, y_t = sess.run([batch_x_t,batch_y_t])
        y_t = Onehot(y_t)
        acc = sess.run(accuracy, feed_dict={X: x_t, Y: y_t, keep_prob: 0.7})
        print("step is:" + str(i), "acc is :" + str(acc))  
    coord.request_stop()
    coord.join(threads)



step is:0 acc is :0.449219
step is:1 acc is :0.390625
step is:2 acc is :0.421875
step is:3 acc is :0.457031
step is:4 acc is :0.429688
step is:5 acc is :0.367188
step is:6 acc is :0.398438
step is:7 acc is :0.390625
step is:8 acc is :0.355469
step is:9 acc is :0.414062
step is:10 acc is :0.371094
step is:11 acc is :0.378906
step is:12 acc is :0.355469
step is:13 acc is :0.417969
step is:14 acc is :0.417969
step is:15 acc is :0.394531
step is:16 acc is :0.335938
step is:17 acc is :0.308594
step is:18 acc is :0.332031
step is:19 acc is :0.421875
step is:20 acc is :0.386719
step is:21 acc is :0.363281
step is:22 acc is :0.390625
step is:23 acc is :0.390625
step is:24 acc is :0.417969
step is:25 acc is :0.382812
step is:26 acc is :0.394531
step is:27 acc is :0.40625
step is:28 acc is :0.390625
step is:29 acc is :0.355469
step is:30 acc is :0.464844
step is:31 acc is :0.371094
step is:32 acc is :0.351562
step is:33 acc is :0.414062
step is:34 acc is :0.402344
step is:35 acc is :0.367188
ste